In [4]:
! pip3 install --user install fuzzywuzzy PyShp geopandas rtree matplotlib cartopy cython cartopy python-Levenshtein

In [16]:
import pandas as pd
import io
import requests
from fuzzywuzzy import process, fuzz
import shapefile
import zipfile
import json
import matplotlib.pyplot as plt
import geopandas as gpd
from cartopy import crs as ccrs
import numpy as np

In [7]:
# read intermediate canonical facilities into dataframe
intermediate_url="https://raw.githubusercontent.com/PTDP/data/main/intermediate_data/geocoded_company_facilities.csv"
intermediate = pd.read_csv(intermediate_url)

hifld_url="https://opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
hifld=pd.read_csv(hifld_url)

In [8]:
# Normalize facility google name

# uppercase
# remove entities before dash IL DOC - BIG MUDDY RIVER CORRECTIONAL CENTER

def clean_facility_internal(str):
    split = str.split("-")
    if (len(split) > 1):
        return split[1]
    return str

intermediate['googlePlaceName'] = intermediate['googlePlaceName'].str.upper();
intermediate['facilityInternal'] = intermediate['facilityInternal'].apply(lambda x: clean_facility_internal(x))

In [9]:
#ratios = process.extract(intermediate['googlePlaceName'][0],hifld['NAME'])
# def fuzzy_matches(googlePlaceName):
    

# highest = process.extractOne(intermediate['googlePlaceName'][0],hifld['NAME'])
# print(highest)

# goal - 

# geosptial join

print(hifld['NAME'][1936])

CULLMAN COUNTY DETENTION CENTER


In [10]:
# Output GeoJSON Data
hifld_shp_url="https://opendata.arcgis.com/datasets/2d6109d4127d458eaf0958e4c5296b67_0.zip?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
local_path = 'hifld.zip'

print('Downloading shapefile...')
r = requests.get(hifld_shp_url)

with open(local_path,'wb') as out: ## Open temporary file as bytes
    out.write(io.BytesIO(r.content).read())                ## Read bytes into file

print("Done")

Done


In [23]:
#print(r.content)
# hifld_geojson = shapefile.Reader(local_path + 'Prison_Boundaries.dbf').__geo_interface__
gdf_intermediate = gpd.GeoDataFrame(intermediate, geometry=gpd.points_from_xy(intermediate.longitude, intermediate.latitude))

# remove null
gdf_intermediate = gdf_intermediate[gdf_intermediate.is_valid]
gdf_intermediate.crs = "EPSG:4326"
gdf_intermediate = gdf_intermediate.to_crs("EPSG:3857")

# buffer in miles around each geocoded point
buffer_length_in_meters = (.5 * 1000) * 1.60934
gdf_intermediate['geometry'] = gdf_intermediate.geometry.buffer(buffer_length_in_meters)

def fuzz_token (arr):
    print(arr)
    return fuzz.token_set_ratio(arr[0], arr[1])


gdf_hifld = gpd.read_file(local_path)

sjoined_facilities = gpd.sjoin(gdf_intermediate, gdf_hifld)

sjoined_facilities["g_name_sim"] = sjoined_facilities.apply(lambda x: fuzz.token_set_ratio(x.googlePlaceName, x.NAME), axis=1)
sjoined_facilities["internal_name_sim"] = sjoined_facilities.apply(lambda x: fuzz.token_set_ratio(x.facilityInternal, x.NAME), axis=1)


def make_column(df):
    return pd.Series(np.random.randn(len(df['g_name_sim'])), index=df.index)

sjoined_facilities['name_override'] =  ""
sjoined_facilities['jurisdiction_override'] = ""
sjoined_facilities['address_override'] = ""
sjoined_facilities['longitude_override'] = ""
sjoined_facilities['latitude_overrride'] = ""
sjoined_facilities['state_override'] = ""
sjoined_facilities['county_override'] = ""
sjoined_facilities['countyFIPS_override'] = ""
sjoined_facilities['HIFLDID_override'] = ""
sjoined_facilities['UCLACovid19ID_override'] = ""
sjoined_facilities['hidden_override'] = ""

# keep only entry with highest google name similarity
sjoined_facilities.sort_values('g_name_sim').drop_duplicates('id', keep='last')
sjoined_facilities.drop('geometry', axis=1).to_csv(r'joined.csv')